In [1]:
from pyabc import (ABCSMC, Distribution, RV,
                   MultivariateNormalTransition)
from pyabc.populationstrategy import AdaptivePopulationSize
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tempfile
import pandas as pd
import scipy as sp
import numpy as np
import subprocess
from io import BytesIO
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))

In [2]:
def simulate(**pars):
    """Wrapper to simulate the myokit model.
    
    Simulates in a subprocess running python2 by passing
    parameters as arguments to (another) wrapper script.
    """
    myokit_python = ("/Users/charles/miniconda3/envs" +
                     "/ion_channel_ABC/bin/python")
    script = "run_channel.py"
    args = [myokit_python, script]
    for p in pars:
        try:
            args.append("-" + str(p))
            args.append(str(pars[p]))
        except:
            print("Error: " +
                  "args is " + str(args))
    re = subprocess.run(args, stdout=subprocess.PIPE)
    d = pd.read_table(BytesIO(re.stdout),
                      delim_whitespace=True,
                      header=0, index_col=0)
    return d

In [3]:
myokit_python = ("/Users/charles/miniconda3/envs" +
                 "/ion_channel_ABC/bin/python")
args = [myokit_python, "get_measurements.py"]
re = subprocess.run(args, stdout=subprocess.PIPE)
measurements = pd.read_table(BytesIO(re.stdout),
                             delim_whitespace=True,
                             header=0, index_col=0)

In [4]:
prior = Distribution(
    g_CaT=RV("uniform", 0, 2),
    E_CaT=RV("uniform", 0, 50),
    p1=RV("uniform", 0, 100),
    p2=RV("uniform", 1, 9),
    p3=RV("uniform", 0, 1),
    p4=RV("uniform", 0, 10),
    p5=RV("uniform", 0, 0.1),
    p6=RV("uniform", 0, 200),
    q1=RV("uniform", 0, 100),
    q2=RV("uniform", 1, 9),
    q3=RV("uniform", 0, 10),
    q4=RV("uniform", 0, 100),
    q5=RV("uniform", 0, 0.1),
    q6=RV("uniform", 0, 100))

In [5]:
def distance(sim, obs):
    dist = 0
    for i in obs.exp.unique():
        try:
            err = np.sum(np.square(obs[obs.exp == i].y - sim[sim.exp == i].y))
        except:
            return float("inf")
        err = pow(err / len(obs[obs.exp == i].y), 0.5)
        err /= np.ptp(obs[obs.exp == i].y)
        dist += err
    return dist

In [6]:
def simulate_pyabc(parameter):
    res = simulate(**parameter)
    return res

In [12]:
abc = ABCSMC(simulate_pyabc, prior, distance,
             population_size=AdaptivePopulationSize(50, 0.8))
abc_id = abc.new(db_path, measurements)

INFO:Epsilon:initial epsilon is 1.9918391437948484
INFO:History:Start <ABCSMC(id=13, start_time=2018-04-19 15:26:43.510924, end_time=None)>


In [13]:
history = abc.run(max_nr_populations=10, minimum_epsilon=0.4)

INFO:ABC:t:0 eps:1.9918391437948484
INFO:ABC:t:1 eps:1.5779436639304452
INFO:Adaptation:Change nr particles 50 -> 247
INFO:ABC:t:2 eps:1.2598272924111102
INFO:Adaptation:Change nr particles 247 -> 387
Process Process-43:
Process Process-41:
Process Process-44:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/multiprocessing/proce

KeyboardInterrupt: 

KeyboardInterrupt
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 23, in work
    new_sim = simulate(new_param)
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/site-packages/pyabc/smc.py", line 556, in eval_one
    model_probabilities)
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/site-packages/pyabc/smc.py", line 405, in _evaluate_proposal
    lambda x: self.distance_function(x, self.x_0), current_eps)
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/site-packages/pyabc/model.py", line 148, in accept
    distance_calculator)
  File "/Users/charles/miniconda3/envs/pyabc/lib/python3.6/site-packages/pyabc